# BLU10 - Part 1 of 3 - Introduction to Recommender Systems
In this specialization, we'll talk about the algorithms that are behind product recommendations on online product platforms like Netflix or Amazon. Typical data sets in these cases contain a huge amount of missing data, more than the actual data points. We'll look into a handy data structure to store these data sets - a sparse matrix - a sophisticated way to represent very large and mostly empty matrices.

<img src="media/book_market.png" width="500">

## 1. Introduction

### 1.1 What are recommender systems? Are they really important?

>"35 percent of what consumers purchase on Amazon and 75 percent of what they watch on Netflix come from product recommendations based on such algorithms." [(link)](https://www.mckinsey.com/industries/retail/our-insights/how-retailers-can-keep-up-with-consumers)


**Recommender Systems** (RS) are commonly used in companies providing a large amount of similar content (e.g. movies/shows on Netflix, songs/podcasts on Spotify, items on Amazon). Since these companies cannot ask all users to rate every single item (Spotify has +50 million tracks and 489 million active users), recommender systems enable the companies to suggest new content both for users which already showed their preferences and to new customers they don't know anything about.

*That seems nice, but why can't we use classification or regression models we already know?*

A recommender system aims to predict the best possible ranking of items for a user. If we are trying to predict a rating, with rating being a numerical value, why can't we consider this as a classical regression or an ordinal classification \*[1] problem? The main aspect of recommender systems (when compared to typical classification/regression problems) is that we are solving a high-sparsity problem (meaning, there are a lot of missing values!). We normally have a severe imbalance for unlabeled data, so we consider this task as a matrix completion problem.

>*[1] Ordinal classification is a classification problem where the labels have an order (e.g. predicting ratings of movies where a higher rating indicates a better movie). Typical classifications problem, such as predicting the weather labels (rain vs sunny vs cloudy) do not have an intrinsic order (sunny is not less or more than cloudy).*

### 1.2 Notorious use cases

**Youtube**: Google-powered, their video recommendation system uses content (e.g. metadata) and user activity data (implicit and explicit) and it is responsible for 60% of video clicks from the home page. For curious minds, here is a link to their [paper](https://www.researchgate.net/profile/Sujoy_Gupta2/publication/221140967_The_YouTube_video_recommendation_system/links/53e834410cf21cc29fdc35d2/The-YouTube-video-recommendation-system.pdf).

**Spotify**: the Swedish company uses a mix of collaborative filtering, NLP, raw audio models and music curators to suggest content for its users. Their recommender system is not fed by explicit ratings but by users' interaction with the software (implicit feedback). 

**Netflix**: by mixing implicit and explicit feedback, the media services provider uses an interface with top-down right-to-left ranking for suggested personalized content. They use your interactions with the service, similarity to other members' tastes and metadata about the titles (e.g. genre, actors) to feed their recommendation system. Simple English explanation from Netflix themselves [here](https://help.netflix.com/en/node/100639).

### 1.3 Types of recommender systems
**Non-personalized systems** are systems where the recommendations are the same for all users or all users within a group. These are useful in situations when we have no information about the user, such as for new users. Typically, we are recommending generally popular items or items liked by similar users, with strategies like:
- Best seller
- Trending hot
- Highest rated
- People who like X also like Y
    
   
**Personalized systems** make tailor-made recommendations for each user. We can use such systems if we have information on the user history.
- Collaborative filtering is based on the user's past behavior. The recommendations can be user-based (recommend items that users similar to me like) or item-based (recommend items similar to those I previously liked).
- Content-based recommendations are based on item features.
- Demographic recommendations are based on user features.
- Personalized learning to rank systems treat recommendation as a ranking problem.

In this BLU, we'll explore non-personalized recommender systems. We'll focus on personalized recommender systems in BLU11 and finally in BLU12 we'll learn about the metrics for RS and create an example workflow.

## 2. Framework

Recommender systems (RS) are software systems that recommend items to users that they might like. We start by learning the main components of an RS. Below is a scheme of a recommender system showing its components and the interactions between them.

The basic model takes into consideration interactions between users and items, while the extended model considers also user and item characteristics.

![Recommender Sytems Framework](./media/recommender_systems_framework.png)

We will refer to this framework frequently throughout the specialization, but for now, let's drill down into each component of the above figure.

### 2.1 Users

The set of *consumers* or people, denoted by $U = \{u_1, u_2, ..., u_m\}$, where the number of users $\left\vert{U}\right\vert = m$.

We reserve the indexing letters $u$ and $v$ to denote generic individual users.

### 2.2 Items

The set of *products* or things, $I = \{i_1, i_2, ..., i_n\}$, with the number of elements $\left\vert{I}\right\vert = n$. 

The indexing letters for items are $i$, $j$ and $l$.

### 2.3 Ratings

Ratings are the *transactions* effectuated by the users or *opinions* provided by the users about the items.

The set of ratings is $R = \{r_{u_1, i_1}, ..., r_{u_m, i_n}\}$, where each rating $r_{u, i}$ corresponds to a user-item pair $(u, i) \in U \times I$.

* Each user $u \in U$ can make at most one rating $r_{u, i}$ for a particular item $i \in I$.
* Any user $u \in U$ is free to rate any number of items $i \in I$, including none.

The ratings are stored in the ratings matrix of size $m\times n$, so the matrix has space for every user-item combination, although typically most ratings are missing. Non-personalized recommender systems usually make exclusive use of the ratings matrix in order to make predictions.

#### 2.3.1 The ratings matrix

The ratings matrix is at the core of any recommender system.

We represent the set of ratings $R = \{r_{u_1, i_1}, ..., r_{u_m, i_n}\}$ as a $U \times I$  matrix - the **ratings matrix** - where the values are the ratings $r_{u, i}$, if they exist:


<img align="center" width="413" height="239" src="./media/ratings_matrix.png">


We represent the ratings which were not recorded as zeros or missing values, enforcing the $U \times I$ shape.

We'll come back to the ratings matrix with more detail in the next notebook.

### 2.4 Profiles

**Profiles** are a collection of objects (users or items in our framework) and their **attributes**.

Consider the set of **item attributes** $A = \{a_1, ..., a_r\}$. These could be characteristics like color, size, ...

We can define **item profiles** $P = \{p_{i_1, a_1}, ..., p_{i_n, a_r}\}$, where $(i, a) \in I \times A$ and values $p_{i, a}$ indicate the presence of attribute $a$ in item $i$. Basically, it's a matrix where the rows are items, the columns are attributes, and the values indicate which attributes has each item.

Let $B$ be a set of **user attributes** (e.g. demographics). We can apply the same reasoning to build **user profiles** $Q = \{p_{u_1, b_1}, ..., p_{u_n, b_r}\}$, where $(u, b) \in U \times B$ and values $p_{u, b}$ indicate the presence of attribute $b$ in user $u$. Again, it's a matrix where the rows are users, the columns are attributes, and the values indicate which attributes has each user.

These matrices are usually used for content-based filtering systems.

### 2.5 User model

As introduced above, RS are in the business of matching users and items. Sometimes it's convenient to have user and item profiles in the same attribute space.

The user model $M$ defines the set $M = \{m_{u_1, a_1}, ..., m_{u_m, a_r}\}$, for $(u, a) \in U \times A$, where $A$ is the set of item attributes. We can represent it as an $m \times r$ matrix where the rows are users and the columns are item characteristics. We get this matrix by combining the ratings matrix (which relates users with items) with the item profiles matrix (which relates items with item attributes).

This matrix is usually used for collaborative or user-based filtering. These are the first of the personalized recommender systems we will learn in the next BLU.

## 3. Recommendations

The whole objective of recommender systems is to fill in the blanks in our ratings matrix and recommend the best possible items to the user. So for each user we want to find, **for the items that they haven't ranked**, the ones that they are most likely to like.

Throughout this specialization, we will learn different ways to predict unseen ratings and we will start with the non-personalized approach that does not take into account specific information about the user.

### 3.1 Prediction step

The RS core computation is to predict the utility of unseen items $i \in I \setminus I_u$ (read set $I$ minus set $I_u$) to a user $u$, where $I_u$  is the subset of items rated by user $u$.

At the core, we want to find a function $f$ that maps user-item pairs into ratings $f : U \times I \to S$ given by $\hat{r}_{u, i} = f(u, i)$ where $S$ is the set of possible ratings. The function should give us values for the missing ratings.

Once we have it, we use the predicted ratings to make recommendations - we recommend the items with the highest predicted ratings. There are two main ways to derive the recommendations from the ratings: top-$K$ and best-item.

### 3.2 Top-*K* items

From the unseen items $L_u \subset (I \setminus I_u$) for the given user, we take the subset with items that have the *k* largest predicted ratings $\hat{r}_{u, i}$.

Optionally, the items in $L_u = \{i_0, i_1, ..., i_k\}$ can be ordered by the predicted rating $\hat{r}_{u, i_0} \geq \hat{r}_{u, i_1} \geq ... \geq \hat{r}_{u, i_k}$, so we can recommend some items more strongly than others.

### 3.3 Best-item

This is just a special case of top-$K$ with $K = 1$. We simply find the item $j$ with the highest predicted rating as $j = \underset{i \space \in \space I \setminus I_u}{\mathrm{argmax}} \space f(u, i)$.

As you see, there are many operations that can be applied to the ratings matrix in order to extract useful information. Most importantly you should have this in the back of your mind...

<img align="center" width="413" height="239" src="media/meme_rmatrix.jpg">

## 4. Context-awareness

Finally, some systems, particularly personalized RS, consider the context in addition to user and item characteristics. Take $C$ as a set of contexts.

The reasoning is that the utility of an item may depend on other variables. A very good camera, for example, may be of lesser utility for a newbie than it is for a pro.

In these cases, we want the prediction function $f$ to use also the context, so $f : U \times I \times C \to S$ given by $\hat{r}_{u, i, c} = f(u, i, c)$.

A particular case of context is time, where $\hat{r}_{u, i, t} = f(u, i, t)$.

As you might imagine, the ratings matrix can become huge in almost no time. If you have 100 users and just 1000 products you get a million user-product combinations with a high probability of having a very sparse matrix (it is very unlikely that each of those 100 users recommended all 1000 products).

Now think about Amazon customer and product base..

In the next notebook, we are going to see how we can map users to products and construct a rating matrix from scratch. See you in a bit!